# Exploration 4. 프로젝트: 멋진 작사가 만들기


라이브러리 불러오기

In [1]:
import os, re, glob
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split


데이터 정제(preprocess_sentence)와 토큰화(tokenize), 그리고 케라스의 LSTM을 사용한 텍스트 생성기 모델(TextGenerator) 그리고 마지막으로 가사를 생성시킬 함수(generate_text)

In [2]:
def preprocess_sentence(sentence):
    sentence=sentence.strip().lower() # 문장의 오른쪽 왼쪽의 공백을 지우고 소문자 변환
    sentence=re.sub(r"([?.!,¿])", r" \1", sentence)  # 특수문자의 양 옆을 각각 한칸 띄우고
    sentence=re.sub(r'[" "]+', " ", sentence) # 여러개의 공백은 하나의 공백으로 바꿉니다.
    sentence=re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) 
    #a-zA-Z?.!,¿가 아닌 모든 특수문자를 하나의 공백으로 바꾼다.
    sentence=sentence.strip() # 문자열에 양끝의 공백을 한번 더 삭제
    sentence=" ".join(['<start> ',sentence,' <end>']) # 문장시작은 <start> 끝에는 <end>를 추가한다.
    return sentence
def tokenize(corpus): 
    tokenizer=tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 단어장 크기12000
        filters=' ',     # preprocessed_sentence
        oov_token="<unk>"
    )
    # corpus를 사용해서 tokenizer 단어장을 만든다. 
    tokenizer.fit_on_texts(corpus)
    
    # 준비한 tokenizer를 사용해서 corpus를 Tensor로 만든다.
    tensor=tokenizer.texts_to_sequences(corpus)
    
    # input data의 시퀀스 크기를 
    #(길이가 짧은 문장 뒤(post)에 padding을 붙여서) 동일하게 만든다.
    # 문장 앞에 패딩을 붙이려면 pre라고 입력하면 됨
    tensor=tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    
    print(tensor, tokenizer)
    return tensor, tokenizer


class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated


### step1. 데이터 다운로드
- 아이펠에서 제공한 가사데이터를 다운받는다.

### Step2. 데이터 읽어오기

In [3]:

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)
print(txt_file_path)
raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])


/aiffel/aiffel/lyricist/data/lyrics/*
데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


### step 3. 데이터 정제

In [4]:
corpus=[]

for sentence in raw_corpus:
    if len(sentence) == 0: continue    # 빈 단어를 corpus에서 뺀다
#    if sentence[-1] == ":" : continue  # 끝에 콜론이 있는 다
        
    # 앞에서 문장부호와 특수문자, 소문자 대문자 처리
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)

tensor, tokenizer=tokenize(corpus)

tensor_=tokenizer.texts_to_sequences(corpus)

tensor=[]

for i in tensor_:
    if len(i)<=15:   # 15개 
        tensor.append(i)
        
tensor=tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

# 마지막토큰은 <end>가 아닌 <pad> 가능성이 높다.
src_input=tensor[:,:-1]
# t<start> 를 잘라서 타겟문장을 만든다.
tgt_input=tensor[:,1:]



[[   2   50    5 ...    0    0    0]
 [   2   17 2647 ...    0    0    0]
 [   2   35    7 ...    0    0    0]
 ...
 [   2  130    5 ...    0    0    0]
 [   2   23   89 ...    0    0    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fa2f4557c10>


### Step 4. 평가 데이터셋 분리

In [5]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,tgt_input, test_size=0.2)

BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256    # 학습시 배치사이즈
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

train_dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
val_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))

train_dataset = train_dataset.shuffle(BUFFER_SIZE)    
val_dataset = val_dataset.shuffle(BUFFER_SIZE)    

train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True)
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)

#dataset

### Step 5. 인공지능 만들기

In [6]:
#Loss
embedding_size=[128,256,512]
hidden_size=[512,1024,2048]
acc={}
for i in embedding_size:
    for j in hidden_size:
        model=TextGenerator(12000+1, i, j) # i: embadding_size, j: hidden_size
        optimizer=tf.keras.optimizers.Adam()
        loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
        model.compile(loss=loss, optimizer=optimizer)
        model.fit(train_dataset, epochs=10, validation_data=(val_dataset))
        acc[f"i-j"]=model

Epoch 1/10
488/488 [==============================] - 41s 75ms/step - loss: 3.7846 - val_loss: 3.3083
Epoch 2/10
488/488 [==============================] - 37s 76ms/step - loss: 3.1825 - val_loss: 3.1017
Epoch 3/10
488/488 [==============================] - 37s 75ms/step - loss: 3.0296 - val_loss: 2.9998
Epoch 4/10
488/488 [==============================] - 37s 76ms/step - loss: 2.9341 - val_loss: 2.9303
Epoch 5/10
488/488 [==============================] - 37s 75ms/step - loss: 2.8579 - val_loss: 2.8742
Epoch 6/10
488/488 [==============================] - 37s 76ms/step - loss: 2.7912 - val_loss: 2.8252
Epoch 7/10
488/488 [==============================] - 37s 76ms/step - loss: 2.7299 - val_loss: 2.7824
Epoch 8/10
 37/488 [=>............................] - ETA: 30s - loss: 2.6559

KeyboardInterrupt: 

In [ ]:
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)